# Accessing Db2 with Go

This lab uses the **go_ibm_db** cli driver, which is already installed in the lab environment. The driver can be used to develop applications that are running on Linux, Mac, or Windows environments. It provides an interface for GoLang to Db2 for z/OS, Db2 for LUW, and Db2 for i.  
  
If you want to use the driver in your own environment you can [download it from GitHub](https://github.com/ibmdb/go_ibm_db). 

The cli driver allows you to run `dynamic SQL statements`. The labs include program examples that run SQL statements with `literals` and also SQL statements with `parameter markers` in the WHERE clauses. 

**Note**: Click on the next cell and press **Shift-Return** to refresh all of the examples in this notebook.

In [27]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Table of Contents

* [Overview of the Db2-Go Lab Environment](#overview)
    * [How to Copy Code and Examples](#copy)
    * [GO Language Environment](#golangenv)
    * [Optional: Exploring the Contents of the Database](#exploredb)
    * [Optional: View the Database Contents Using Db2 Jupyter Extensions](#exploredbext)
* [Connect to Database SAMPLE](#connect)
* [Retrieve a Single Row from a Table](#select_single)
* [Execute a Query and Fetch the Result Set](#fetch)
* [Prepare and Execute with Parameter Markers](#markers)
* [Retrieve the Column Names from the Result Set](#fetch_columns)
* [Create a Table](#cre_tab)
* [Insert a Row](#insert)
* [Insert Multiple Rows](#insert_prep)
* [Update a Row](#update)
* [Delete Rows](#delete)
* [Perform Multiple Changes](#update_multiple)
* [Perform Multiple Changes in a Unit of Work](#update_multiple_uow)
* [Additional sample Programs](#solutions)

    * [count3.go](#count3.go)

<a id='overview'></a>
## Overview of the Db2-Go Lab Environment

<a id='copy'></a>
### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.

In [28]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
Sample commands are found in cells like this.
</pre>
</div>

The entire contents of the text in the cell will be automatically copied when you click on the cell. The color of the background will change color briefly to indicate that the copy has completed. To paste commands into a terminal window, use the key combination **Control-Shift-v**. 

Code blocks that are meant to be placed into an editor will have a **COPY** button found at the bottom of the text. You can manually select the text you want to copy, or use the **COPY** button to place the entire contents onto the clipboard. 

<!--sample.code-->
```go
If (code == sample) then
   place onto clipboard
else
   ignore
end if
```

In [29]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('sample.code')">Copy</button> 

<a id='golangenv'></a>
### GO Language Environment

The following variables should be set in your Linux environment:
```
DB2HOME='/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver'
CGO_CFLAGS=-I$DB2HOME/include
CGO_LDFLAGS=-L$DB2HOME/lib
LD_LIBRARY_PATH='/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver/lib'
```

For OSX you require the following environment setting.
```bash
DYLD_LIBRARY_PATH=$DYLD_LIBRARY_PATH:/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver/lib
```

Before you start, make sure that the environment is set up correctly:
- Open a Linux shell
   ![](./media/terminal-selected.png)


- If you are not already logged on as user `db2pot`, switch to user `db2pot` (password: `123qwe123`). 
<br><p>**Note:** Execute all Go program examples as user **db2pot**.    

In [30]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1000 onmousedown="window.clipline(1000)" onmouseup="window.reset(1000)">
su - db2pot
</pre>
</div>

- Verify that the above listed environment variables are set in your environment. To check the value of an environment variable, for example `LD_LIBRARY_PATH`, run the following command: 
```
export|grep LD_LIBRARY_PATH
```

In [31]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1100 onmousedown="window.clipline(1100)" onmouseup="window.reset(1100)">
export | grep DB2HOME
</pre>
</div>

- The environment variables are defined in file `.profile` in directory `/home/db2pot`. Use the following command to show the contents of the file:
```
more /home/db2pot/.profile
```

- To define an environment variable use the `export` command. For example, to define `LD_LIBRARY_PATH` use the following command:
```
export LD_LIBRARY_PATH='/home/db2inst1/sqllib/lib64:/home/db2inst1/sqllib/lib64/gskit:/home/db2inst1/sqllib/lib32'
```

In [32]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1200 onmousedown="window.clipline(1200)" onmouseup="window.reset(1200)">
more /home/db2pot/.profile
</pre>
</div>

Execute all program examples from the Linux shell. To execute a program, take the following steps:

- Create directory `Labs` in the home directory of user db2pot (**Note:** This may already be done for you):  

In [33]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1300 onmousedown="window.clipline(1300)" onmouseup="window.reset(1300)">
cd /home/db2pot
mkdir Labs
</pre>
</div>

In [34]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1400 onmousedown="window.clipline(1400)" onmouseup="window.reset(1400)">
cd /home/db2pot/Labs
leafpad hello_world.go &
</pre>
</div>

- Copy and paste the program code to the leafpad editor. The `leafpad` editor can be found in the task bar if you need to access it outside of a command line.
   ![](./media/leafpad-selected.png)
   The first line of each program example shows the program name that should be used when you create the corresponding program file. For example, you would save the following program code in a file named ```hello_world.go```.

<!--hello_world.go-->
```go
// hello_world.go
package main
import "fmt"
func main() {
    fmt.Println("Hello world")
}
```

In [35]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('hello_world.go')">Copy</button> 

- Click the `Save` button and save the program to directory ```/home/db2pot/Labs``` with the proper program name (`hello_world.go` in this example).


- Open a Shell window and execute the program as follows:

```
cd /home/db2pot/Labs
go run hello_world.go
```

In [36]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1600 onmousedown="window.clipline(1600)" onmouseup="window.reset(1600)">
go run hello_world.go
</pre>
</div>

Now you are ready to [start with the first Db2-Go tutorial](#connect) that explains how to connect to the SAMPLE database using GO language. Optionally, you can also explore the sample database as described in the following two sections. This is not a prerequisite for the Db2-Go tutorials.

<a id='exploredb'></a>
### Optional: Exploring the Contents of the Database

You use database **SAMPLE** for the Db2-Go tutorials. The database is already installed in the lab environment. You can check [the schema description of the database](https://www.ibm.com/support/knowledgecenter/en/SSEPGG_11.1.0/com.ibm.db2.luw.apdv.samptop.doc/doc/r0001094.html).
Below is a list of the tables that are used in the labs together with the corresponding number of rows and columns.

| Tables                 | Number of Rows          | Number of Columns  |
|:---------------------- |:------------------------|:-------------------|
|ACT|18|3|
|DEPARTMENT|14|5|
|EMPLOYEE  |42|14|
|EMPPROJACT|73|6|
|ORG|8|5|
|PROJECT|20|9|
|ROLES|11|2|
|SALES|41|4|
|STAFF|3|7|

You can explore the database using either the Db2 command line, Visual Studio Code, or the Db2 Jupyter notebook extensions. To explore the database from the Db2 command line perform the following steps.

1) Open a shell window (click the *terminal* icon in the task bar below the Linux desktop) 
   ![](./media/terminal-selected.png)

   Use the following technique to make it easier to switch between a Jupyter notebook and the command line. When you open up the terminal window, right click on the title bar and select `Always on Top`. This will keep the terminal window on top of your Jupyter notebook so you can easily move between the two environments.
   
   ![](./media/alwaysontop.png)
   
   Remember to use `Shift-Control-v` when copying text into a command window.


2) Switch to user **db2inst1** (password: **db2inst1**). You can use the **db2pot** user to issue Db2 commands, but you will need to connect as user **DB2INST1**. 

In [37]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=100 onmousedown="window.clipline(100)" onmouseup="window.reset(100)">
su - db2inst1
</pre>
</div>

3) Start Db2. The database may already be started, but in the event that you receive an error when connecting in the next step, issue the following command.  

In [38]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=200 onmousedown="window.clipline(200)" onmouseup="window.reset(200)">
db2start
</pre>
</div>

4) Connect to database SAMPLE:  

In [39]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=300 onmousedown="window.clipline(300)" onmouseup="window.reset(300)">
db2 connect to sample
</pre>
</div>

5) Show the tables in schema DB2INST1:  

In [40]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=400 onmousedown="window.clipline(400)" onmouseup="window.reset(400)">
db2 "select substr(tabname,1,15) from syscat.tables where tabschema='DB2INST1'"
</pre>
</div>

6) Show the table definition of one of the tables, for example, table ACT:  

In [41]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=500 onmousedown="window.clipline(500)" onmouseup="window.reset(500)">
db2 "describe table db2inst1.act"
</pre>
</div>

7) Retrieve the rows from one of the tables, for example, table ACT:  

In [42]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=600 onmousedown="window.clipline(600)" onmouseup="window.reset(600)">
db2 "select * from db2inst1.act"
</pre>
</div>

8) When you are done, enter the command `exit` to return back to the `db2pot` environment.

In [43]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=700 onmousedown="window.clipline(700)" onmouseup="window.reset(700)">
exit
</pre>
</div>

<a id='exploredbext'></a>
### Optional: View the Database Contents Using Db2 Jupyter Extensions
The following code will load the Db2 Magic commands required to run SQL against Db2 from within a Jupyter Notebook.

In [ ]:
%run db2.ipynb
%run connection.ipynb

List the tables in schema DB2INST1:

In [ ]:
%sql -grid select substr(tabname,1,15) from syscat.tables where tabschema='DB2INST1'

Describe table ACT:

In [ ]:
%sql describe table act

View the contents of table ACT:

In [ ]:
%sql -grid select * from act

<a id='connect'></a>
## Connect to Database SAMPLE

You start with a simple GO program that connects to the `SAMPLE` database. It imports the following packages which are required to deploy the Db2 driver API:
```go
import _ "github.com/ibmdb/go_ibm_db"  
import "database/sql"  
```
**Note:** The underscore before the package `github.com/ibmdb/go_ibm_db` is required. It ensures that the `init` function of the package is executed and package-level variables are created.

The function `sql.Open()` is executed to setup a database connection. It requires the driver name `go_ibm_db` and the connection string `con` as input parameters. The connection string specifies hostname, port number, database name, user name, and password. 
If `sql.Open()` was executed successfully, the database handle `db` is initialized. Otherwise it will be set to `nil`. Before the program terminates it calls function `db.Close()`. It closes the database connection and cleans up the database handle.  
  
Execute the `connect.go` program example from the shell as described in the previous section.

<!--connect.go-->
```go
// connect.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

func main(){
        con := "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"
        db, err:=sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
        }
        fmt.Println("Success!")
        db.Close()
}
```

In [44]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('connect.go')">Copy</button> 

Now adjust the code to make it more handy to use. Since the database handle `db` is always required to execute a Db2 API function, define the connection related variables `db`, `err`, and `con` outside of function `main()`. This makes sure you can access the database handle in all functions that are defined in package `main`.  

Also put the `db.Open()` call into a separate function `connect()`. 

Finally, make sure that function `db.Close()` is automatically called when function `main()` terminates. Put the `defer` keyword in front of the statement. This makes sure that `db.Close()` is automatically executed as soon as a return statement is executed anywhere in function `main()`.

The following program code contains all modifications that are described above. Execute `connect2.go` from the shell.

<!--connect2.go-->
```go
// connect2.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }
    
        fmt.Println("Success!")
}
```

In [45]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('connect2.go')">Copy</button> 

<a id='select_single'></a>
## Retrieve a Single Row from a Table

In the next example, you use function `db.QueryRow()` to run a query that is expected to return at most one row. If there multiple rows in the query result set,  the function will only access the first row and discard the rest.  
  
You use the following select statement in this example:  
  
```sql
select count(*) statement from act
```

This statement always returns exactly one row and returns the number of records in table ACT.

The `Scan()` function copies the columns from the current row into the values pointed. Since we expect a single integer value in the query result set, we define variable `count` of type `int32` and pass a pointer to that variable into function `scan()`.  
  
Execute program `count_in_main.go` from the shell.

<!--count_in_main.go-->
```go
// count_in_main.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        var count int32
        err = db.QueryRow("SELECT count(*) FROM act").Scan(&count)
        if err != nil {
                fmt.Println(err)
                return
        }
        fmt.Println("Rowcount=",count)
}
```

In [46]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count_in_main.go')">Copy</button> 

In the next example, the call to function `db.QueryRow()` is moved into a separate function `count()`. This function expects a tablename as a parameter. It prints the row count of the table and returns parameter `err`. 
This allows us to check in `main()` if an error occurred in function `count()`.

<!--count2.go-->
```go
// count2.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func count(tabname string) error { 
        var count int32
        err = db.QueryRow("SELECT count(*) FROM "+tabname).Scan(&count)
        if err != nil {
                fmt.Println(err)
                return err
        }       
        fmt.Printf("Table \"%s\" contains %d rows.\n",tabname,count)
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        count("ACT")
        count("DEPARTMENT")
        count("EMPLOYEE")
        count("null")
        count("ORG")
}
```

In [47]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count2.go')">Copy</button> 

You see that one of the table names in the list is invalid ("`null`"). In this case the program just prints an error message and continues to process. You could also do additional error handling in function `main()`. For example, you could remove the `Printf()` statement in function `count()` and then invoke `count()` in function `main()` as follows:  
```go
if count("ACT") != nil { fmt.Printf("Error in function count()") }
```
Try to rewrite program `count2.go`. Create a new program `count3.go` that performs the following steps:
- Stores the tablenames in a dynamic array
- Iterates over the array
- Passes each table name to function `count()`
- Checks the return value of function `count()` and does the error handling in function `main()`.

You will find the corresponding program `count3.go` at the end of this notebook.

<a id='fetch'></a>
## Execute a Query and Fetch the Result Set

Next, you learn how to run a `SELECT` statement that returns more than one row. In this example, we use the following select statement:  
  
 ```sql
 select firstnme, lastname, job from employee where job='MANAGER'
 ```  
  
Function `db.Query()` prepares and executes the `SELECT` statement. It is also possible to separate the preparation and execution of a SQL statement. This can help to reduce the overhead for statement preparation and is shown in a later example.

Function `rows.Next()` iterates over the result set and prepares the next result row for reading with the `Scan()` api. Since the `SELECT` statement returns three values in each row of the result set, we define variables `a`,`b`, and `c` and pass their address to function `Scan()`. The function copies the columns from the current row into variables `a`,`b`,and `c`.

Execute program `fetch_rows.go` from the shell. 

<!--fetch_rows.go-->
```go
// fetch_rows.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select firstnme, lastname, job from employee where job='MANAGER'")
        if err != nil {
                return
        }
        // make sure that the "rows" handle is released when main returns
        defer rows.Close()

        // iterate over all rows in the query result
        var a,b,c string
        for rows.Next() {
                err = rows.Scan(&a,&b,&c)
                if err != nil{
                        fmt.Println(err)
                        return
                }
                fmt.Printf("%-10s %-10s %-10s\n",a,b,c)
        }
}
```

In [48]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('fetch_rows.go')">Copy</button> 

<a id='markers'></a>
## Prepare and Execute with Parameter Markers

In case you want to execute the same query with different filter conditions you can prepare the statement once and then execute it multiple times. This allows you to reuse the prepared statement and reduces the overhead of statement preparation. To do this you need to use a parameter markers (?) as shown in the example below:
  
 ```sql
 "select firstnme, lastname, job, workdept from employee where workdept = ?"
 ```  

The statement is prepared by executing function `db.Prepare()`. The function returns a handle `st` to the prepared statement:
```go
st, err := db.Prepare("select firstnme, lastname, job, workdept from employee where workdept = ?")
```

Function `st.Query()` prepares and executes the SQL statement. We use statement handle `st` to reference the prepared statement. We also have to pass the appropriate number of parameters to the function. Since we prepared the statement with one parameter marker, we pass one parameter `dept` to the function:
```go
rows,err := st.Query(dept)
```

Execute program `parameter_markers.go` from the shell. 

<!--parameter_markers.go-->
```go
// parameter_markers.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        // prepare the statement once with a parameter marker
        st, err := db.Prepare("select firstnme, lastname, job, workdept from employee where workdept = ?" )
        if err !=nil {
                fmt.Println("Error in Prepare: ")
                fmt.Println(err)
                return
        }
        // execute the statement multiple times and use a different
        // work department in the where clause for each query execution
        departments := []string{"A00","B01","C01","D11","D21","E11","E21"}
        for _,dept := range departments{
                fmt.Printf("\nSelect records for department '%s'\n", dept)
                rows,err := st.Query(dept)
                if err != nil {
                        fmt.Println("Error in Query: ")
                        fmt.Println(err)
                        return
                }

                // iterate over all rows in the query result
                for rows.Next() {
                        var a,b,c,d string
                        err = rows.Scan(&a,&b,&c,&d)
                        if err != nil{
                                fmt.Println(err)
                                return
                        }
                fmt.Printf("%-10s %-10s %-10s %-10s\n",a,b,c,d)
                }
                rows.Close()
        }
}
```

In [26]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('parameter_markers.go')">Copy</button> 

<a id='fetch_columns'></a>
## Retrieve the Column Names from the Result Set

Db2 provides the option to rename table columns. So the names of columns may change over time. Therefore, it can be useful to retrieve the current column names from a table. 

To retrieve the column names, we use an SQL query that selects all columns from the table and retrieves one row only:  
  
 ```sql
 select * from employee fetch first 1 row only
 ```  
   
   
We use function `db.Query()` to execute that query. The function returns a handle to the query result (`rows`):

 ```go
 rows,err := db.Query("select * from employee fetch first 1 row only")
 ```  
  
We then use handle `rows` to retrieve the column names. Function `rows.Columns()` stores the column names in a dynamically created array:  
  
 ```go
 cols, err := rows.Columns()
 ```
  
We can use function `Printf()` to print the whole array at once:  
  
 ```go
 fmt.Printf("%v\n",cols) 
 ```
  
Alternatively, we can use the `range` operator to iterate over the array and print each element on a separate line. There are different ways to use the range operator:  
 ```go  
 for _,name := range cols { ... }
 for idx,name := range cols { ... }
 ```  

In our example, we use the first form which only retrieves the elements of the array `cols`. Alternatively, you can also retrieve the index value of each array element. 



Execute program `get_columns.go` from the shell. 

<!--get_columns.go-->
```go
// get_columns.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select * from employee fetch first 1 row only")
        if err != nil {
                fmt.Printf("db.Query(): error!")
                return
        }
        // make sure that the "rows" handle is released when main returns
        defer rows.Close()

        cols, err := rows.Columns()
        fmt.Println("Number of columns: ",len(cols))
        // print the whole array at once
        fmt.Printf("%v\n",cols)
        // print each column name on a separate line
        for _,name := range cols {
                fmt.Printf("%s\n",name)
        }
}
```

In [51]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('get_columns.go')">Copy</button> 

<a id='cre_tab'></a>
## Create a Table

The previous sections covered the execution of queries and used function ```Query()``` which is expected to return data.

In the following labs you create tables and change data using the following types of SQL statements:
* CREATE TABLE
* INSERT
* UPDATE
* DELETE


The Db2-GO API provides function ```Exec()``` to execute SQL statements that do not return a result set. This function can either prepare and execute a statement in one single step or you can first prepare a statement and then use ```Exec()``` to execute the statement.

The next sample program creates a new table using the following SQL statement:
```sql
create table LINEITEM(ID int,NAME varchar(20),QTY int)
```

In this example, the statement is prepared and executed in one step. In case the table already exists, function ```Exec()``` will return error ```SQL0601``` and the program will only print the error message and terminate.

Execute program ```cre_tab.go``` from the shell.


<!--cre_tab.go-->
```go
// cre_tab.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        _,err:=db.Exec("create table LINEITEM(ID int,NAME varchar(20),QTY int)")
        if err != nil{
                fmt.Println("Error:")
                fmt.Println(err)
                return
        }
        fmt.Println("TABLE CREATED")
}
```


In [52]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('cre_tab.go')">Copy</button> 

You can check if the table was successfully created by running the following commands from the shell (the password of user *db2inst1* is *db2inst1*):
```
su - db2inst1
db2start
db2 connect to sample
db2 describe table lineitem
```

<a id='insert'></a>
## Insert a Row

Next, you insert a single row into table ```LINEITEM``` with the following SQL statement:
```sql
insert into lineitem values (99,'Flowers',5)
```
Since we want to execute only one single ```INSERT``` statement, we keep the code simple and prepare and execute the statement in one single step:
```go
_,err:=db.Exec("insert into lineitem values (99,'Flowers',5)")
```

Execute program ```insert.go``` from the shell.


<!--insert.go-->
```go
// insert.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        _,err:=db.Exec("insert into lineitem values (99,'Flowers',5)")
        if err != nil{
                fmt.Println("Error:")
                fmt.Println(err)
                return
        }
        fmt.Println("Row inserted.")
}
```

In [53]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('insert.go')">Copy</button> 

You can check if the data was successfully inserted by running the following commands from the shell (the password of user db2inst1 is db2inst1):
```
su - db2inst1
db2start
db2 connect to sample
db2 "select * from lineitem"
```

<a id='insert_prep'></a>
## Insert Multiple Rows

In case you want to insert multiple records into a table, you can prepare the INSERT statement first and then execute it multiple times as shown in the next example. We use a SQL statement which contains parameter markers as follows:
```sql
insert into lineitem values (?,?,?)
```

We first execute function ```Prepare()``` which returns handle ```st``` to the prepared statement:
```go
st, err := db.Prepare("insert into lineitem values (?,?,?)")
```

Then we create a dynamic array ```lineitems```, iterate over the elements in the array, assign the value of each array element to ```item```, and its index number to ```idx```:
```go
lineitems := []string{"Shirt","Bicycle","Laptop","Coffee","Burger","Watch"}
for idx,item := range lineitems {
```

Finally, we call function  ```Exec()``` and pass parameter values for each of the three parameter markers. We use a constant value 5 for column ```QTY``` (quantity):
```go
_,err = st.Exec(idx,item,5)
```

Execute program ```insert_prep.go``` from the shell.

<!--insert_prep.go-->
```go
// insert_prep.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        // prepare the statement once with a parameter markers
        st, err := db.Prepare("insert into lineitem values (?,?,?)" )
        if err !=nil {
                fmt.Println("Error in Prepare: ")
                fmt.Println(err)
                return
        }

        lineitems:= []string{"Shirt","Bicycle","Laptop","Coffee","Burger","Watch"}
        for idx,item := range lineitems{
                _,err = st.Exec(idx,item,5)
                if err != nil{
                        fmt.Println("Error:")
                        fmt.Println(err)
                        return
                }
                fmt.Println("Row inserted.")
        }
}
```


In [54]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('insert_prep.go')">Copy</button> 

<a id='update'></a>
## Update a Row

The next program example updates one of the records in table LINEITEM. It uses an `UPDATE` statement which contains parameter markers as follows:
```sql
update lineitem set qty=? where id=?
```

As in the previous example, the program first executes function ```Prepare()``` and then calls function  ```Exec()``` and passes the appropriate parameter values for each of the two parameter markers that are used in the SQL statement. 

Before you execute `update.go` check the current content of table `LINEITEM` from the shell:
```
su - db2inst1
db2 connect to sample
db2 "select * from lineitem"
```

The output should look like this:

|ID|NAME|QTY|
|:----------------------|:------------------------|:-------------------|
|99|Flowers|5|
|0|Shirt|5|
|1|Bicycle|5|
|2|Laptop|5|
|3|Coffee|5|
|4|Burger|5|
|5|Watch|5|

Now execute program `insert_prep.go ` from the shell.

<!--update.go-->
```go
// update.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        // prepare the statement with parameter markers
        st, err := db.Prepare("update lineitem set qty=? where id=?")
        if err !=nil {
                fmt.Println("Error in Prepare: ")
                fmt.Println(err)
                return
        }

        id := 0
        qty := 3
        _, err = st.Exec(id, qty)
        if err != nil{
                fmt.Println("Error:")
                fmt.Println(err)
                return
        }
        fmt.Println("Row updated.")
}

```

In [55]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('update.go')">Copy</button> 

The program should run without error. Now check again the contents of table `LINEITEM`. You notice that lineitem ID=0 still has QTY=5. Try to find the error and fix the program code so that the QTY field of lineitem ID=0 is properly updated and contains value 3.

Hint: The content of table `LINEITEM` after execution of program `update.go` looks like below. Obviously, the wrong item was updated.

|ID|NAME|QTY|
|:----------------------|:------------------------|:-------------------|
|99|Flowers|5|
|0|Shirt|5|
|1|Bicycle|5|
|2|Laptop|5|
|3|Coffee|0|
|4|Burger|5|
|5|Watch|5|

Solution: You need to change the order of the parameters in the call to function `Exec()`.

Change the corresponding program line as follows:
```
_, err = st.Exec(qty,id)
```
The parameters must match the order of the parameters in the SQL statement:
```sql
update lineitem set qty=? where id=?
```

<a id='delete'></a>
## Delete Rows

The next program example deletes multiple records from table `LINEITEM`. It uses a `DELETE` statement which contains a parameter marker as follows:
```sql
delete from lineitem where name=?
```

The statement is prepared with the following function call:
```go
st, err := db.Prepare("delete from lineitem where name=?")
```


The names of the items to be deleted are stored in a dynamic array `lineitems`. The program iterates over the array as shown below: 
```go
lineitems := []string{"Shirt","Coffee"}
for _,item := range lineitems{
    _,err = st.Exec(item)
```

Execute program `delete.go ` from the shell. 

<!--delete.go-->
```go
// delete.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        st, err := db.Prepare("delete from lineitem where name=?")
        if err !=nil {
                fmt.Println("Error in Prepare: ")
                fmt.Println(err)
                return
        }

        lineitems:= []string{"Shirt","Coffee"}
        for _,item := range lineitems{
                _,err = st.Exec(item)
                if err != nil{
                        fmt.Println("Error:")
                        fmt.Println(err)
                        return
                }
                fmt.Println("Item deleted:")
                fmt.Println(item)
        }
}
```

In [56]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('delete.go')">Copy</button> 

<a id='update_multiple'></a>
## Perform Multiple Changes

By default, SQL statemets that modify data are immediately commited after they are executed. It is also possible to perform multiple changes in one unit of work as you will see in a later example. 

The following sample program can be used to verify the default behaviour which auto commits each statement. The program retrieves each record from table `LINEITEM` and assigns a new quantity value to the record. 

It retrieves the records with the following select statement:
```sql
select * from lineitem
```
The values of each retrieved record are first stored into variables `id`, `name`, and `qty`. A record is modified with the following update statement:
```sql
update lineitem set qty=? where id=?
```

When you execute the program it will randomly select a new quantity value and assign this value to each record in the table. The new quantity value is based on the current time, i.e. with each execution of the program a different quantity value will be used.

After a record was updated the program waits for one second before it continues to process the next record. This allows you to interrupt the program while the records are updated. 

Now perform the following steps:
- Execute program `update_all.go` from the shell and wait until it has completed all updates.
- Check the content of table `lineitem` by running the following commands from the shell: 

``` 
su - db2inst1
db2 connect to sample
db2 "select * from lineitem"
```
- Execute program `update_all.go` again and interrupt the program after it has updated the first two records. To interrupt the program type `Ctrl-C` in the shell window where you started the program.
- Check the content of table `lineitem`. 

You see that some records have been updated with the new quantity value while other records have not been updated yet. In many cases, this is not the desired behaviour. In transactional systems you have to ensure that either all SQL statements of a transaction are performed or none of them. In the next lab we will modify program `update_all.go` to implement this behaviour.

<!--update_all.go-->
```go
// update_all.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
    "time"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select * from lineitem")
        if err != nil {
                return
        }
        defer rows.Close()

        var currentTime int64
        currentTime = time.Now().Unix()
        newqty := currentTime & 0x00000000000000FF
        fmt.Println("Quantity:", newqty)

        var id int32
        var name string
        var qty int32

        for rows.Next() {
                err = rows.Scan(&id,&name,&qty)
                if err != nil{
                        fmt.Println(err)
                        return
                }
                fmt.Printf("Fetched one row:\n")
                fmt.Printf("%-5d %-10s %-5d\n",id,name,qty)
                time.Sleep(1*time.Second)
            
                fmt.Printf("Updating row with new quantity value: %d\n",newqty)
                _,err = db.Exec("update lineitem set qty=? where id=?",newqty,id)
                if err != nil{
                        fmt.Println(err)
                        return
                }

        }
}
```

In [57]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('update_all.go')">Copy</button> 

<a id='update_multiple_uow'></a>
## Perform Multiple Changes in a Unit of Work

The following program example extents the previous program example and makes sure that all update statements are performed in one unit of work (UoW). This is achieved by calling function `Begin()` before the update operations start and by calling function `Commit()` after the updates are completed.

Function `Begin()`returns a handle `uow` to the unit of work as follows:
```go
uow, err := db.Begin()
```

This handle must be used in all following SQL calls that should be part of this particular unit of work. Therefore, we execute the update statements as follows:
```go
_,err = uow.Exec("update lineitem set qty=? where id=?",newqty,id)
```

We also use handle `uow` to execute function `Commit()` as follows:
```go
err = uow.Commit()
```

Now perform the following steps:

- Execute program `update_all_uow.go` from the shell and wait until it has completed all updates.
- Check the content of table lineitem by running the following commands from the shell:
```
su - db2inst1
db2 connect to sample
db2 "select * from lineitem"
```
- Execute program `update_all_uow.go` again and interrupt the program after it has updated the first two records. To interrupt the program type Ctrl-C in the shell window where you started the program.
- Check the content of table lineitem. You see that no records have been updated.

As explained above, you can use functions `Begin()` and `Commit()` to perform multiple changes as one atomic transaction. This ensures that either all changes or none of the changes are performed.


<!--update_all_uow.go-->
```go
// update_all_uow.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
    "time"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select * from lineitem")
        if err != nil {
                return
        }
        defer rows.Close()

        var currentTime int64
        currentTime = time.Now().Unix()
        newqty := currentTime & 0x00000000000000FF
        fmt.Println("Quantity:", newqty)

        var id int32
        var name string
        var qty int32

        // Begin Unit of Work (UoW)
        uow, err := db.Begin()
        if err != nil {
                fmt.Println(err)
                return
        }
        for rows.Next() {
                err = rows.Scan(&id,&name,&qty)
                if err != nil{
                        fmt.Println(err)
                        return
                }
                fmt.Printf("Fetched one row:\n")
                fmt.Printf("%-5d %-10s %-5d\n",id,name,qty)
                time.Sleep(1*time.Second)
            
                fmt.Printf("Updating row with new quantity value: %d\n",newqty)
                _,err = uow.Exec("update lineitem set qty=? where id=?",newqty,id)
                if err != nil{
                        fmt.Println(err)
                        return
                }

        }
        // End Unit of Work
        err = uow.Commit()
        if err != nil {
                fmt.Println(err)
                return
        }
}
```


In [58]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('update_all_uow.go')">Copy</button> 

<a id='solutions'></a>
## Additional sample programs

<a id='count3.go'></a>
### count3.go

<!--count3.go-->
```go
// count3.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func count(tabname string) error {
        var count int32
        err = db.QueryRow("SELECT count(*) FROM "+tabname).Scan(&count)
        if err != nil {
                return err
        }
        fmt.Printf("Table \"%s\" contains %d rows.\n",tabname,count)
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        tabs := []string{"ACT","EMPLOYEE","null","ROLES","SALES","ORG","STAFF"}
        for index, tab := range tabs {
                fmt.Printf("%d\n", index)
                err = count(tab)
                if err != nil { fmt.Println(err) }
        }
}
```

In [50]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count3.go')">Copy</button> 

#### Credits: IBM 2019, Andreas Christian [achristian@de.ibm.com]